# chapter 2.

- to capture skeleton; 

In [1]:
import cv2
import mediapipe as mp


# 포즈 추출을 위한 모듈 불러오기
mp_pose = mp.solutions.pose  
mp_draw = mp.solutions.drawing_utils


# 포즈 객체 생성 (기본 옵션으로 사용)
pose = mp_pose.Pose()

# cap = cv2.VideoCapture(0)  # 웹캠 켜기
cap = cv2.VideoCapture('fall.mp4') #to capture from the mp4 file;


while True:
    ret, frame = cap.read()
    if not ret:
        break

    
    # BGR(OpenCV) -> RGB(MediaPipe)
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # 포즈 추정
    results = pose.process(img_rgb)

    # 관절 좌표가 있으면 그리기
    if results.pose_landmarks:
        mp_draw.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    cv2.imshow("Pose Estimation", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


- to obtain the coordinates of the left hip;

In [2]:
import cv2
import mediapipe as mp


# 포즈 추출을 위한 모듈 불러오기
mp_pose = mp.solutions.pose  
mp_draw = mp.solutions.drawing_utils


# 포즈 객체 생성 (기본 옵션으로 사용)
pose = mp_pose.Pose()

# cap = cv2.VideoCapture(0)  # 웹캠 켜기
cap = cv2.VideoCapture('fall.mp4') #to capture from the mp4 file;


while True:
    ret, frame = cap.read()
    if not ret:
        break

    
    # BGR(OpenCV) -> RGB(MediaPipe)
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # 포즈 추정
    results = pose.process(img_rgb)

    # 관절 좌표가 있으면 그리기
    if results.pose_landmarks:
        mp_draw.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        
        #to get the coordinates(x,y,z) of each skeleton;
        landmarks = results.pose_landmarks.landmark

        # for example; left hip;
        left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
        print(f'coordinate of left-hip; \n{left_hip}') 

        left_hip_x = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x
        print(f'----- the x-coordinate of left-hip; {left_hip_x}')
        left_hip_y = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y
        print(f'----- the y-coordinate of left-hip; {left_hip_y}')
        left_hip_z = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].z
        print(f'----- the z-coordinate of left-hip; {left_hip_z}')        
        print('\n\n')

        
    cv2.imshow("Pose Estimation", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

coordinate of left-hip; 
x: 0.379414409
y: 0.668970585
z: 0.00134751911
visibility: 0.999681592

----- the x-coordinate of left-hip; 0.3794144093990326
----- the y-coordinate of left-hip; 0.6689705848693848
----- the z-coordinate of left-hip; 0.001347519108094275



coordinate of left-hip; 
x: 0.379412472
y: 0.672365129
z: 0.00166928058
visibility: 0.999675632

----- the x-coordinate of left-hip; 0.3794124722480774
----- the y-coordinate of left-hip; 0.672365128993988
----- the z-coordinate of left-hip; 0.0016692805802449584



coordinate of left-hip; 
x: 0.379475296
y: 0.675447285
z: 0.00161803735
visibility: 0.999672532

----- the x-coordinate of left-hip; 0.37947529554367065
----- the y-coordinate of left-hip; 0.6754472851753235
----- the z-coordinate of left-hip; 0.0016180373495444655



coordinate of left-hip; 
x: 0.379777908
y: 0.676679075
z: 0.00154391187
visibility: 0.999674857

----- the x-coordinate of left-hip; 0.3797779083251953
----- the y-coordinate of left-hip; 0.6766790

- to obtain values of each skeleton;

- to detect falls using a rule-base approach;

In [3]:
import cv2
import mediapipe as mp
import time

mp_pose = mp.solutions.pose
mp_draw = mp.solutions.drawing_utils
pose = mp_pose.Pose()


prev_y = None
fall_threshold = 10
fall_detected = False



cap = cv2.VideoCapture('fall.mp4')

output_path = 'fall_detection.avi'
out = None



while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(img_rgb)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        mp_draw.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)


        
        left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP]
        y = int(left_hip.y * frame.shape[0])

        
        #to define falls if dy is less threshold;
        if prev_y is not None:
            dy = y - prev_y
            if dy > fall_threshold:
                fall_detected = True
        prev_y = y


    
    if fall_detected:
        cv2.putText(frame, 'Fall Detected', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 
                    1, (0,0,255), 3)

    if out is None:
        height, width, _ = frame.shape
        out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'XVID'), 10, (width, height))

    out.write(frame)
    cv2.imshow('frame', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
if out:
    out.release()
cv2.destroyAllWindows()

# quiz;
- fall detection;
- enhance the stability of fall detection by applying multiple rules;

In [4]:
import cv2
import mediapipe as mp
import time
import math

mp_pose = mp.solutions.pose
mp_draw = mp.solutions.drawing_utils
pose = mp_pose.Pose()

cap = cv2.VideoCapture('fall.mp4')

output_path = 'fall_detection.avi'
out = None

prev_y = None
fall_detected = False
frame_count = 0
fall_frames = 0

fall_threshold = 15   # hip movement
angle_threshold = 40  # degrees
confirm_frames = 8    # how many frames to confirm fall

def calculate_angle(p1, p2):
    """Calculate angle between two points (for vertical posture check)."""
    dx = p2[0] - p1[0]
    dy = p2[1] - p1[1]
    angle = math.degrees(math.atan2(dy, dx))
    return angle

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(img_rgb)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        mp_draw.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Extract key joints
        left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP]
        right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP]
        left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER]
        right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER]
        nose = landmarks[mp_pose.PoseLandmark.NOSE]

        # Convert to pixel coordinates
        hip_y = int((left_hip.y + right_hip.y)/2 * frame.shape[0])
        shoulder_y = int((left_shoulder.y + right_shoulder.y)/2 * frame.shape[0])
        nose_y = int(nose.y * frame.shape[0])

        # Rule 1: Sudden drop in hip
        if prev_y is not None:
            dy = hip_y - prev_y
            if dy > fall_threshold:
                fall_frames += 1
            else:
                fall_frames = max(0, fall_frames-1)
        prev_y = hip_y

        # Rule 2: Body orientation (angle between shoulders and hips)
        hip_x = int((left_hip.x + right_hip.x)/2 * frame.shape[1])
        shoulder_x = int((left_shoulder.x + right_shoulder.x)/2 * frame.shape[1])

        angle = calculate_angle((hip_x, hip_y), (shoulder_x, shoulder_y))
        if abs(angle) < angle_threshold:  # close to horizontal
            fall_frames += 1

        # Rule 3: Head close to hips
        if abs(nose_y - hip_y) < 50:  # pixels, adjust per video resolution
            fall_frames += 1

        # Confirm fall if multiple frames satisfy conditions
        if fall_frames >= confirm_frames:
            fall_detected = True

    if fall_detected:
        cv2.putText(frame, 'Fall Detected!', (50, 50), cv2.FONT_HERSHEY_SIMPLEX,
                    1, (0, 0, 255), 3)

    # Save and display
    if out is None:
        height, width, _ = frame.shape
        out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'XVID'), 10, (width, height))

    out.write(frame)
    cv2.imshow('frame', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
if out:
    out.release()
cv2.destroyAllWindows()
